In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import PiecewiseAffineTransform, warp
from skimage import data
import os
import pandas as pd
from PIL import Image
import imageio

images=os.listdir("D:\\desktop\\simulation new\\data set\\FGNET\\FGNET\\images\\1.gray scale")
images=np.asarray(images)

points=os.listdir("D:\\desktop\\simulation new\\data set\\FGNET\\FGNET\\txtpts")
points=np.asarray(points)

df = pd.read_csv (r'D:\\desktop\\simulation new\\data set\\FGNET\\FGNET\\txtpts\\013A18.csv',sep=' ',header=None)
dst=np.asarray(df,dtype=np.float32)
xmin=58
xmax=353
ymin=210
ymax=468
bd2=np.array([[xmin,ymin],[xmax,ymin],[xmin,ymax],[xmax,ymax]])
dst = np.concatenate((bd2, dst), axis=0)

for i in range(len(points)):
    impath="D:\\desktop\\simulation new\\data set\\FGNET\\FGNET\\images\\1.gray scale\\"+images[i]
    ptpath="D:\\desktop\\simulation new\\data set\\FGNET\\FGNET\\txtpts\\"+points[i]


    image = Image.open(impath) 
    image = np.asarray(image, dtype=np.float32)
    rows, cols = image.shape[0], image.shape[1]

    image2 = Image.open("D:\\desktop\\simulation new\\data set\\FGNET\\FGNET\\images\\1.gray scale\\013A18.jpg") 
    image2 = np.asarray(image2, dtype=np.float32)
    rows2, cols2 = image2.shape[0], image2.shape[1]

    df = pd.read_csv (ptpath,sep=' ',header=None)
    src=np.asarray(df,dtype=np.float32)
    #bd=np.array([[0,0],[rows,0],[0,cols],[rows,cols]])
    bd=np.array([[0,0],[cols,0],[0,rows],[cols,rows]])
    new = np.concatenate((bd, src), axis=0)
    src=new

    tform = PiecewiseAffineTransform()
    tform.estimate(dst, src)

    out_rows = 600
    out_cols = 600
    out = warp(image, tform, output_shape=(out_rows, out_cols))
    out=out[ymin:ymax,xmin:xmax]

    
    size = (29, 29)
    img= Image.fromarray(out)
    img = img.resize(size)
    small = np.array(img.getdata())
    a=np.reshape(small, size)
    b=a / 255.0
    mean=np.mean(b)
    sd=np.std(b)
    b=(b-mean)/sd
    
    os.chdir("D:\\desktop\\simulation new\\data set\\FGNET\\FGNET\\small29n")
    np.savetxt(images[i][:-4]+".csv", b, delimiter=",")
